# Section 5

In [2]:
import cvxpy as cp
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

class color:
    PURPLE = '\033[95m'
    CYAN = '\033[96m'
    DARKCYAN = '\033[36m'
    BLUE = '\033[94m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    RED = '\033[91m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
    END = '\033[0m'

Implement the mean absolute error:
$$
MAE = \frac{1}{N}\sum_{i=1}^N |y_i-x_i^\top\theta|
$$

In [3]:
import math
def get_MAE(theta, X, y):
    ypred = X@theta.T
    mae = np.average(np.abs(ypred - y), axis=0)
    
    # compare to MAE from sklearn:
    mae_sklr = mean_absolute_error(X@theta.T, y)
    assert(math.isclose(mae, mae_sklr))
    return mae

## Question 7.2
Implement the problem from Question 7.1 (Use the `GLPK` solver)

In [4]:
# Function to solve LPs from previous question 
def solve_LP(X, Y, lambda_, k):
    d = X.shape[1]
    N = X.shape[0]

    # auxiliary variables:
    Beta = cp.Variable((N, 1))
    b = cp.Variable((d, 1))
    alpha = cp.Variable((1,1))

    # variables to solve:
    theta = cp.Variable((1, d))

    # linear program:
    prob = cp.Problem( cp.Minimize(k * alpha + cp.sum(Beta) + lambda_ * cp.sum(b)), 
                      [
        alpha + Beta >= 1/k * (Y - X @ theta.T),
        alpha + Beta >= -1/k * (Y - X @ theta.T),
        Beta >= 0,
        -b <= theta,
        theta <= b
    ])

    # solve LP:
    prob.solve(solver=cp.GLPK)
    theta_opt = theta.value
    alpha_opt = b.value
    
    opt_value = prob.value
    dual_value = prob.constraints[0].dual_value
    return theta_opt,  opt_value, dual_value 

In [5]:
# Hyperparameters:
lambda_ = np.logspace(-5, -1, 50, base = 10)
lambda_

array([1.00000000e-05, 1.20679264e-05, 1.45634848e-05, 1.75751062e-05,
       2.12095089e-05, 2.55954792e-05, 3.08884360e-05, 3.72759372e-05,
       4.49843267e-05, 5.42867544e-05, 6.55128557e-05, 7.90604321e-05,
       9.54095476e-05, 1.15139540e-04, 1.38949549e-04, 1.67683294e-04,
       2.02358965e-04, 2.44205309e-04, 2.94705170e-04, 3.55648031e-04,
       4.29193426e-04, 5.17947468e-04, 6.25055193e-04, 7.54312006e-04,
       9.10298178e-04, 1.09854114e-03, 1.32571137e-03, 1.59985872e-03,
       1.93069773e-03, 2.32995181e-03, 2.81176870e-03, 3.39322177e-03,
       4.09491506e-03, 4.94171336e-03, 5.96362332e-03, 7.19685673e-03,
       8.68511374e-03, 1.04811313e-02, 1.26485522e-02, 1.52641797e-02,
       1.84206997e-02, 2.22299648e-02, 2.68269580e-02, 3.23745754e-02,
       3.90693994e-02, 4.71486636e-02, 5.68986603e-02, 6.86648845e-02,
       8.28642773e-02, 1.00000000e-01])

In [43]:
from sklearn.datasets import load_diabetes
diabetes = load_diabetes()

In [44]:
# 75% split:
X, X_test, Y, Y_test = train_test_split(diabetes['data'], 
                                        np.expand_dims(diabetes['target'], 1), 
                                        test_size=0.25, random_state=0)
# Add bias column to data:
X = np.concatenate((X, np.ones((X.shape[0], 1))), axis=1)
X_test = np.concatenate((X_test, np.ones((X_test.shape[0], 1))), axis=1)

e_train, e_val, thetas, opt_val = [], [], [], []


In [8]:

# Cross-validation for 50 values of lambda:
for l in lambda_:
    theta_opt, opt_value, dual_value = solve_LP(X, Y, l, math.floor(0.75 * X.shape[0]))
    thetas.append(theta_opt)
    opt_val.append(opt_value)
    
    # evaluate on training set:
    e_train.append(get_MAE(theta_opt, X, Y))
    # evaluate on validation set:
    e_val.append(get_MAE(theta_opt, X_test, Y_test))
    
# take hyperparameter with smallest validation error:
best_lambda = lambda_[np.argmin(e_val)]
best_theta = thetas[np.argmin(e_val)]
best_value = opt_val[np.argmin(e_val)]

print('---Optimal values--')
print(f'Optimal lambda: {best_lambda}')
print(f'Optimal value: {best_value}')
print(f'Optimal theta:\n {best_theta}')

---Optimal values--
Optimal lambda: 0.004094915062380423
Optimal value: 69.85032423411252
Optimal theta:
 [[ -51.25036737 -256.59298168  256.59298168  256.59298168  134.15685618
  -256.59298168 -256.59298168  256.59298168  256.59298168  249.77586814
   151.12806892]]


In [9]:
print(color.BOLD + 'Training Results' + color.END)
print('MAE: {}'.format(get_MAE(best_theta, X, Y)))
print('\n')
print(color.BOLD + 'Test Results' + color.END)
print('MAE: {}'.format(get_MAE(best_theta, X_test, Y_test)))

Training Results
MAE: [46.42683699]


Test Results
MAE: [43.17935492]
